In [1]:
import csv
import numpy as np

MJUPITER = 1.898e27
MEARTH = 5.972e24
RJUPITER = 69.911e6
REARTH = 6371e3

In [2]:
class System:
    def __init__(self, sysid=None):
        self.sysid = sysid

In [3]:
my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/exoarchive_ipac_confirmed_200507.csv'  #reading in file

raw_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        raw_data.append(row)
        
KEYS = raw_data[77]
VALUES = raw_data[78:]

def get_key(keyname, keys = KEYS, values = VALUES):
    i = keys.index(keyname)
    arr = []
    for row in values:
        arr.append(row[i])
    return arr

In [4]:
def is_number(num):
    '''
    Checks if a piece of data (in string format) is a number or not.
    '''
    try:
        float(num)
        return True
    except ValueError:
        return False 

def convert_masses(mass_arr):
    new_mass_arr = []
    for m in mass_arr:
        if is_number(m):
            m = str(float(m)*MJUPITER/MEARTH)
        new_mass_arr.append(m)
    return new_mass_arr

def convert_radii(radii_arr):
    new_radii_arr = []
    for r in radii_arr:
        if is_number(r):
            r = str(float(r)*RJUPITER/REARTH)
        new_radii_arr.append(r)
    return new_radii_arr
        

d = {}
d['starname'] = get_key('pl_hostname')
d['rp'] = convert_radii(get_key('pl_radj'))  #get radius of planets (Earth radius)
d['rs'] = get_key('st_rad')  #get radius of star (solar radii)
d['mp'] = convert_masses(get_key('pl_bmassj'))  #get mass of planets (in Earth masses)
                                #mass can be expressed as Msin(i), where i is the inclination
d['ms'] = get_key('st_mass')  #get mass of stars (in solar masses)
d['p'] = get_key('pl_orbper')  #get planet's period (in days)
d['disc_method'] = get_key('pl_discmethod')  #get planet's discovery method
d['source'] = ['Exoarchive']*len(d['disc_method'])
d['update'] = get_key('rowupdate')
d['temp'] = get_key('st_teff')

print('Number of Exoarchive planets: %d' %len(d['starname']))

Number of Exoarchive planets: 4152


In [5]:
my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/clean_kepler_catalogue.csv'
cks_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        cks_data.append(row)
        
KEYS = cks_data[0]
VALUES = cks_data[1:]

def get_key(keyname,keys=KEYS,values=VALUES):
    '''
    keyname = (string) of column definition, see CKS documentation
    '''
    kid = keys.index(keyname)
    
    dummy = []
    for row in values:
        dummy.append(row[kid])
    
    return dummy

d['starname'] = np.concatenate((d['starname'], get_key('id_starname')))
d['rp'] = np.concatenate((d['rp'], get_key('iso_prad')))
d['rs'] = np.concatenate((d['rs'], get_key('iso_srad')))
d['ms'] = np.concatenate((d['ms'], get_key('iso_smass')))
d['p'] = np.concatenate((d['p'], get_key('koi_period')))
d['disc_method'] = np.concatenate((d['disc_method'], ['Transit']*len(get_key('koi_period'))))

masses = np.load('C:/Users/djhoo/Documents/maiasaurus-master/catalogues/cks_pred_masses_oct9.npy')[0]
d['mp'] = np.concatenate([d['mp'], masses])
d['source'] = np.concatenate((d['source'], ['cks']*len(get_key('koi_period'))))
d['update'] = np.concatenate((d['update'], ['']*len(get_key('koi_period'))))
d['temp'] = np.concatenate((d['temp'], get_key('koi_steff')))

print('Number of CKS planets: %d' %len(get_key('id_starname')))

Number of CKS planets: 864


In [6]:
my_file = 'C:/Users/djhoo/Documents/maiasaurus-master/catalogues/kepler_dr25.csv'
cks_data = []
with open(my_file) as infile:
    reader = csv.reader(infile)
    
    for row in reader:
        cks_data.append(row)
        
KEYS = cks_data[0]
VALUES = cks_data[1:]

def get_key(keyname,keys=KEYS,values=VALUES):
    '''
    keyname = (string) of column definition, see first line
    '''
    kid = keys.index(keyname)
    
    dummy = []
    for row in values:
        dummy.append(row[kid])
    
    return dummy

names = get_key('kepoi_name')
for i in range(len(names)):
    names[i] = names[i][0:names[i].find('.')]
rp = get_key('koi_prad')
rs = get_key('koi_srad')
mp = ['']*len(get_key('koi_srad'))
ms = get_key('koi_smass')
p = get_key('koi_period')
disc_method = ['Transit']*len(get_key('koi_period'))
source = ['dr_25']*len(get_key('koi_period'))
update = ['']*len(get_key('koi_period'))
temp = get_key('koi_steff')
count = 0

for i in range(len(names)):
    if d['mp'][d['starname']==names[i]].tolist() != ['']*len(d['starname'][d['starname']==names[i]]):
        pass
    else:
        np.append(d['starname'], names[i])
        np.append(d['rp'], rp[i])
        np.append(d['mp'], mp[i])
        np.append(d['ms'], ms[i])
        np.append(d['p'], p[i])
        np.append(d['disc_method'], disc_method[i])
        np.append(d['source'], source[i])
        np.append(d['update'], update[i])
        np.append(d['temp'], temp[i])
        count = count + 1

print('Number of DR25 planets: %d' %count)

Number of DR25 planets: 8660


In [7]:
mixed = ['GJ 357', 'Kepler-25', 'Kepler-41', 'Kepler-48', 'Kepler-65', 'Kepler-68', 'Kepler-82', \
        'HD 3167', 'Kepler-20', 'Kepler-56', 'WASP-47', 'Kepler-19']

for i in range(len(d['starname'])):
    if d['starname'][i] in mixed:
        d['disc_method'][i] = 'Mixed'

In [8]:
'''
The data entries are the following:

- Starname - name of host star
- Rp - radius of exoplanet (in Earth radii)
- Mp - mass of exoplanet (in Earth masses)
- P - period of exoplanet (in days)
- Rs - radius of host star (in solar radii)
- Ms - mass of host star (in solar masses)
- Temp - temperature of host star (in K)
- Detection Method - detection method (Radial Velocity, Transit, or Mixed)
- Source - original text file
'''

output_heading = ['Starname', 'Rp', 'Mp', 'P', 'Rs', 'Ms', 'Detection Method', 'Source', 'Update']
myData = [output_heading]
for i in range(len(d['starname'])):
    arr = []
    arr.append(d['starname'][i])
    arr.append(d['rp'][i])
    arr.append(d['mp'][i])
    arr.append(d['p'][i])
    arr.append(d['rs'][i])
    arr.append(d['ms'][i])
    arr.append(d['disc_method'][i])
    arr.append(d['source'][i])
    arr.append(d['update'][i])
    myData.append(arr)
outputFile = open('C:/Users/djhoo/Documents/maiasaurus-master/catalogues/file_master.csv', 'w', newline='')
with outputFile:
    writer = csv.writer(outputFile)
    writer.writerows(myData)
print('Writing Complete!')
    

Writing Complete!
